<h1 style="align: center;font-size: 18pt;">Multiprocess Lambdas</h1>
<hr style="height:2.5px">

This tutorial shows the user how to using the standard Python library `multiprocessing` to multiprocess lambda values in `BICePs`. 

<hr style="height:2.5px">

In [1]:
import os, sys, pickle
import numpy as np
import biceps
import multiprocessing as mp

BICePs - Bayesian Inference of Conformational Populations, Version 2.0


### Data and Output Directories 

In [5]:
energies = np.loadtxt('cineromycin_B/cineromycinB_QMenergies.dat')*627.509  # convert from hartrees to kcal/mol
energies = energies/0.5959   # convert to reduced free energies F = f/kT
energies -= energies.min()  # set ground state to zero, just in case
states = len(energies)
top = 'cineromycin_B/cineromycinB_pdbs/0.fixed.pdb'
print(f"Possible input data extensions: {biceps.toolbox.list_possible_extensions()}")
data = biceps.toolbox.sort_data('cineromycin_B/J_NOE')
res = biceps.toolbox.list_res(data)
print(f"Input data: {biceps.toolbox.list_extensions(data)}")
outdir = 'results_test'
biceps.toolbox.mkdir(outdir)

Possible input data extensions: ['H', 'Ca', 'N', 'J', 'noe', 'pf']
Input data: ['J', 'noe']


### Parameters

In [6]:
nsteps=10000000
print(f"nSteps of sampling: {nsteps}")
maxtau = 1000
n_lambdas = 3
lambda_values = np.linspace(0.0, 1.0, n_lambdas)
ref = ['uniform', 'exp']
uncern = [[0.05, 20.0, 1.02], [0.05, 5.0, 1.02]]

nSteps of sampling: 10000000


### Multiprocessing Lambda values #######

In [7]:
def mp_lambdas(Lambda):
    ensemble = biceps.Ensemble(Lambda, energies, top)
    ensemble.initialize_restraints(input_data=data, ref_pot=ref,
            uncern=uncern, gamma=[0.2, 5.0, 1.02])
    sampler = biceps.PosteriorSampler(ensemble.to_list())
    sampler.sample(nsteps=nsteps, verbose=False)
    sampler.traj.process_results(outdir+'/traj_lambda%2.2f.npz'%(lam))
    outfilename = 'sampler_lambda%2.2f.pkl'%(lam)
    fout = open(os.path.join(outdir, outfilename), 'wb')
    pickle.dump(sampler, fout)
    fout.close()
    print('...Done.')

### Note that the output from each lambda will be returned to stdout as soon as the job has completed...

In [8]:
# Check the number of CPU's available
print("Number of CPU's: %s"%(mp.cpu_count()))
p = mp.Pool(processes=n_lambdas) # knows the number of CPU's to allocate
print(f"Number of processes: {n_lambdas}")
jobs = []
for lam in lambda_values:
    process = p.Process(target=mp_lambdas, args=(lam,))
    jobs.append(process)
    jobs[-1].start() # Start the processes
    active_processors = [jobs[i].is_alive() for i in range(len(jobs))]
    if (len(active_processors) == mp.cpu_count()-1) and all(active_processors) == True:
        while all(active_processors) == True:
            active_processors = [jobs[i].is_alive() for i in range(len(jobs))]
        inactive = int(np.where(np.array(active_processors) == False)[0])
        jobs[inactive].terminate()
        jobs.remove(jobs[inactive])
for job in jobs:
    job.join() # will wait until the execution is over...
p.close()

Number of CPU's: 6
Number of processes: 3

Accepted 71.40438999999999 % 


Accepted [32.73672 29.90617 29.90617  8.7615 ] % 

...Done.

Accepted 66.00635 % 


Accepted [32.73587 29.9568  29.9568   3.31368] % 


Accepted 64.30934 % 


Accepted [32.74651 29.97059 29.97059  1.59224] % 

...Done.
...Done.


<h6 style="align: justify;font-size: 12pt"># <span style="color:red;">NOTE</span>: The following cell is for pretty notebook rendering</h6>

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../../../theme.css", "r").read()
    return HTML(styles)
css_styling()